In [1]:
import os
import shutil

import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
# import tensorflow_text as text
import keras

from keras import Input
from keras.layers import TextVectorization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import (
    GRU,
    Conv1D,
    Dense,
    Embedding,
    Flatten,
    Lambda,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

print(tf.__version__)

2023-02-15 18:45:49.803540: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-15 18:45:50.679453: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-02-15 18:45:50.679547: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

2.11.0


In [2]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

2023-02-15 18:45:54.051117: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-15 18:45:54.065091: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-15 18:45:54.066892: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-15 18:45:54.069199: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [3]:
#read data
data = pd.read_csv("../data/patent_rand100k.csv")
#shuffle data
# data = data.sample(frac=1)

In [4]:
# # data.groupby('ipc_section').count()
# dataSort = data.query('(ipc_section == "A")').sample(n=1000)
# dataSort = pd.concat([dataSort,data.query('(ipc_section == "B")').sample(n=1000)])
# dataSort = pd.concat([dataSort,data.query('(ipc_section == "C")').sample(n=1000)])
# dataSort = pd.concat([dataSort,data.query('(ipc_section == "D")').sample(n=1000)])
# dataSort = pd.concat([dataSort,data.query('(ipc_section == "E")').sample(n=1000)])
# dataSort = pd.concat([dataSort,data.query('(ipc_section == "F")').sample(n=1000)])
# dataSort = pd.concat([dataSort,data.query('(ipc_section == "G")').sample(n=1000)])
# dataSort = pd.concat([dataSort,data.query('(ipc_section == "H")').sample(n=1000)])
# # data=data.sample(50000)
# # data=dataSort

In [5]:
len(data)

100000

In [6]:
#one hot encode target
# classes = [[0,1,2,3,4,5,6,7][source] for source in dataSort.ipc_section]
classes = [{"A":0,"B":1,"C":2,"D":3,"E":4,"F":5,"G":6,"H":7}[source] for source in data.ipc_section]
target = to_categorical(classes)
#print target
# target[2002]
len(target)

100000

In [11]:
BATCHSIZE = 100

inputs = Input(shape=(512,),dtype=tf.float32,name="InputString")
# x = Dense(256,activation="relu")(inputs)
# x = Dense(256,activation="relu",kernel_regularizer=tf.keras.regularizers.L2(0.02))(inputs)
# x = tf.keras.layers.Dropout(0.2)(x)
outputs = Dense(8,activation="softmax")(inputs)

model = keras.Model(inputs=inputs,outputs=outputs,name="modeltest")
model.summary()
model.compile(optimizer="adam", loss="categorical_crossentropy",metrics=["Accuracy"])

Model: "modeltest"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputString (InputLayer)    [(None, 1)]               0         
                                                                 
 dense_2 (Dense)             (None, 8)                 16        
                                                                 
Total params: 16
Trainable params: 16
Non-trainable params: 0
_________________________________________________________________


In [12]:
modelhistory = model.fit(
    embed(data.abstract[:80000]),
    target[:80000], 
    epochs=50,
    batch_size=BATCHSIZE, 
    validation_split=0.2
    # validation_data=(data.text[6000:],target[6000:]),
    # callbacks=[EarlyStopping(patience=10)]
)

ValueError: When input_signature is provided, all inputs to the Python function must be convertible to tensors:
  inputs: (
    0        The invention is a compound of R or S enantiom...
1        A frozen section embedding box, comprising a b...
2        The present invention relates to a method for ...
3        The invention relates to a preparation method ...
4        The application discloses an unstructured high...
                               ...                        
79995    The utility model relates to a kind of coating...
79996    PURPOSE:To confirm the braking positively in s...
79997    An adjustable device is disclosed comprising t...
79998    The invention discloses a processing fixture f...
79999    The present invention relates to a providing a...
Name: abstract, Length: 80000, dtype: object)
  input_signature: (
    TensorSpec(shape=<unknown>, dtype=tf.string, name=None)).

In [ ]:
embed(data.abstract[:30000])

2023-02-15 18:49:14.883194: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1191600000 exceeds 10% of free system memory.
2023-02-15 18:49:15.395051: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1190880000 exceeds 10% of free system memory.
2023-02-15 18:49:15.841801: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1191600000 exceeds 10% of free system memory.
2023-02-15 18:49:16.331335: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1191600000 exceeds 10% of free system memory.
2023-02-15 18:49:18.370223: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2383200000 exceeds 10% of free system memory.


In [ ]:
pd.DataFrame(modelhistory.history)[["loss","val_loss"]].plot(title="BS="+ str(BATCHSIZE) +" | USE | 256 dense")
pd.DataFrame(modelhistory.history)[["Accuracy","val_Accuracy"]].plot(title="BS="+ str(BATCHSIZE) +" | USE | 256 dense")